In [2]:
'''
************************************************
** COL759 Cryptography and Computer Security
** Assignment 2
** @Author: Aman Bhardwaj
** @Entry No. 2019SIY7580
** @Date: 27 Feb 2020
************************************************
'''

#pip install C:\Users\Aman\Downloads\gmpy2-2.0.8-cp37-cp37m-win_amd64.whl
import gmpy2 as mp
import numpy as np
import math

blockLen = 0
charsToNum = {}
numToChars = {}
charSpace = 29
ca_signature = "this key has been signed by ca signature"

def readFile(fname):
    '''Read File from the directory'''
    try:
        text = ""
        with open(fname,'r') as openFile:
            t = openFile.read().replace('\n','')
        openFile.close()
        t = t.replace("'", "")
        t = t.replace("?", "")
        #print(t)
        return t.lower()
    except:
        print("No such file found in directory. Please check")
        return

def writeFile(fname, text):
    '''write to File from the directory'''
    try:
        with open(fname,'w') as openFile:
            openFile.write(text)
        openFile.close()
        return
    except:
        print("No such file found in directory. Please check")
        return

def charsDictGen():  
    '''#form dictionary of chars a-z and map them to 0-25 numbers'''
    charsDict = {"a":None}
    init = 'a'
    for i in range(0,26):
        charsDict[init] = i
        init = chr(ord(init) + 1)
    charsDict[' '] = 26
    charsDict[','] = 27
    charsDict['.'] = 28
    return charsDict

def numbersDictGen():  
    '''#form dictionary of 0-25 numbers and map them to chars a-z '''
    numsDict = {0:None}
    init = 'a'
    for i in range(0,26):
        numsDict[i] = init
        init = chr(ord(init) + 1)
    numsDict[26] = ' '
    numsDict[27] = ','
    numsDict[28] = '.'
    return numsDict


def generateStrongPrime(bits):
    '''Generates strong prime using Gordon's Algorithm'''
    b = round(bits/2)
    s = mp.next_prime(2**b)
    b += 1
    t = mp.next_prime(2**b)
    r = mp.next_prime(t)
    p0 = mp.mul(mp.powmod(mp.mpz(s), r-1, r), s) -1
    prp = mp.next_prime(p0 + 2*mp.mul(r,s))
    
    #Verify if generated prime is strong
    notStrongPrime = True
    prp = mp.next_prime(2**bits)
    count = 0
    while notStrongPrime:
        if mp.is_strong_bpsw_prp(prp):
            notStrongPrime = False
        elif not mp.is_strong_bpsw_prp(prp):
            prp = mp.next_prime(prp)
    return prp

def generateStrongPriem(bits):
    '''Generates strong prime using Gordon's Algorithm'''
    b = round(bits/2)
    s = mp.next_prime(2**b)
    b += 1
    t = mp.next_prime(2**b)
    r = mp.next_prime(t)
    p0 = mp.mul(mp.powmod(mp.mpz(s), r-1, r), s) -1
    prp = mp.next_prime(p0 + 2*mp.mul(r,s))
    return prp

def getMsgBlocks(text, l):
    '''Returns list of message blocks of length l'''
    msgBlocks = []
    tempBlock = ""
    
    for char in text:
        tempBlock += char
        if(len(tempBlock) == l):
            msgBlocks.append(tempBlock)
            tempBlock = ""
    msgBlocks.append(tempBlock) 
    return msgBlocks

def enVigenere(text, key):
    '''Encrypt the given message from vigenere cipher.'''
    
    msgBlocks = getMsgBlocks(text, len(key))
    enMsg = ""
    for block in msgBlocks:
        for i in range(len(block)):
            enMsg += numToChars[(charsToNum[block[i]] + charsToNum[key[i]])%charSpace]
    
    return enMsg

def deVigenere(text, key):
    '''Decrypt the given message from vigenere cipher.'''
    msgBlocks = getMsgBlocks(text, len(key))
    deMsg = ""
    for block in msgBlocks:
        for i in range(len(block)):
            deMsg += numToChars[(charsToNum[block[i]] - charsToNum[key[i]])%charSpace]
    
    return deMsg

def getPQ(b):
    '''returns p and q for rsa for generating n = p*q of min length "b" bits'''
    b = round(b/2)
    p = generateStrongPrime(b)
    
    b += 1
    q = generateStrongPrime(b)
    return p,q


def rsaKeyGen(bits):
    '''RSA Private and Public Key Generator'''
    p, q = getPQ(bits)
    n = mp.mul(p,q)
    phi = mp.mul(p-1, q-1)
    e = 2**445 + 1
    notCoprime = True
    while notCoprime:
        if mp.gcd(e, phi) == 1:
            notCoprime = False
        else:
            e += 2
    
    d = mp.invert(e, phi)
    #print("phi\n", phi)
    #print(mp.t_mod(mp.mul(e,d), phi) == 1)
    return e, n, d

def signDigitally(text):
    '''Sign a Document Digitally by CA by his private key'''
    pk = readFile("public_directory/public_" + "ca" + ".txt").split()
    sk = readFile("secret_" + "ca" + ".txt").split()
    
    n = mp.mpz(pk[1])
    d = mp.mpz(sk[0])
    blockLenDsc = 40
    #print("Block Length", blockLenDsc, type(blockLenDsc))
                
    msg = 0
    for i in range(blockLenDsc):
        msg += mp.mul(charsToNum[text[i]], mp.mpz(charSpace)**(blockLenDsc -1 -i))     

    eMsg = mp.powmod(msg, d, n) #sign by CA Secret Key
    
    return eMsg

def signDigital_CA(key):
    '''Sign a Document Digitally by CA by his private key'''
    sk = readFile("secret_" + "ca" + ".txt").split()
    
    n = mp.mpz(sk[1])
    d = mp.mpz(sk[0])
    
    return mp.powmod(key, d, n)

def unsignDigital_CA(key):
    pk = readFile("public_directory/public_" + "ca" + ".txt").split()
    e = mp.mpz(pk[0])
    n = mp.mpz(pk[1])
    
    return mp.powmod(key, e, n)

def deCaSignature(text):
    '''recover sign of CA by his public key'''
    pk = readFile("public_directory/public_" + "ca" + ".txt").split()
    
    e = mp.mpz(pk[0])
    n = mp.mpz(pk[1])
    blockLenDsc = 40
    
    dSign = mp.powmod(mp.mpz(text), e, n) #Check signature Validity by CA Pbblic key

    remainder = dSign
    m = []
    decryptedSign = ""
    for i in range(blockLenDsc):
        quot, remainder = mp.t_divmod(remainder, mp.mpz(charSpace)**(blockLenDsc -1 -i))
        m.append(numToChars[mp.t_mod(quot, charSpace)])
    decryptedSign += ''.join(m)
        
    return decryptedSign

def publishCaSignedKeys():
    '''Publish CA Signed Public Keys in Public Directory and Send Secret Keys to respective users'''
    e_ca, n_ca, d_ca = rsaKeyGen(1027)
    writeFile("public_directory/public_ca.txt", str(e_ca) + " " + str(n_ca))
    writeFile("secret_ca.txt", str(d_ca) + " " + str(n_ca))
    

    e_a, n_a, d_a = rsaKeyGen(1024)
    e_a_dsca = signDigital_CA(e_a)
    n_a_dsca = signDigital_CA(n_a)
    d_a_dsca = signDigital_CA(d_a)
    writeFile("public_directory/public_a.txt", str(e_a_dsca) + " " + str(n_a_dsca) + " " + str(e_a) + " " + str(n_a))
    writeFile("secret_a.txt",str(d_a_dsca) + " " + str(n_a_dsca) + " " + str(d_a) + " " + str(n_a))

    e_b, n_b, d_b = rsaKeyGen(1024)
    e_b_dsca = signDigital_CA(e_b)
    n_b_dsca = signDigital_CA(n_b)
    d_b_dsca = signDigital_CA(d_b)
    writeFile("public_directory/public_b.txt", str(e_b_dsca) + " " + str(n_b_dsca) + " " +  str(e_b) + " " + str(n_b))
    writeFile("secret_b.txt",str(d_b_dsca) + " " + str(n_b_dsca) + " " + str(d_b) + " " + str(n_b))
    
    return

def calcBlockSize(n):
    '''Calculates Block size by the value of n'''
    a = mp.mpz(n)
    match = False
    r = 0
    while mp.mpz(charSpace)**r < a:
        r += 1
    return r

def validateCASignature(dSig):
    '''Validate CA Signature. Returns True of verified'''
    x = unsignDigital_CA(mp.mpz(dSig[0])) #operate CA's public key
    y = unsignDigital_CA(mp.mpz(dSig[1])) #operate CA's public key
    
    a = mp.mpz(dSig[2]) # a=e for encryption a = d for decryption
    b = mp.mpz(dSig[3])
    if x == a and y == b: #if the numbers e/d and n match with the unsigned_ca values of the themselves. Key is valid
        return True
    else:
        return False
    
    
def enRSA(text, sender, receiver, operation):
    '''RSA Encryption'''
    
    operator = 0
    
    if(operation == "d"):
        sk = readFile("secret_" + sender + ".txt").split()
        if validateCASignature(sk):
            print("\nCA SIGNATURE Verified. Hence Key is Valid")
        else:
            print("\nCA SIGNATURE Verification Failed. Hence Key is invalid")
            return
        print("\nEncrypting by Sender's[{}] Secret Key.".format(sender))
        d = unsignDigital_CA(mp.mpz(sk[0].strip()))
        n = unsignDigital_CA(mp.mpz(sk[1].strip()))
        N = n
        operator = d
    elif(operation == "e"):
        pk = readFile("public_directory/public_" + receiver + ".txt").split()
        if validateCASignature(pk):
            print("\nCA SIGNATURE Verified. Hence Key is Valid")
        else:
            print("\nCA SIGNATURE Verification Failed. Hence Key is invalid")
            return
        print("\nEncrypting by Receiver's[{}] Public Key.".format(receiver))
        e = unsignDigital_CA(mp.mpz(pk[0].strip()))
        n = unsignDigital_CA(mp.mpz(pk[1].strip()))
        N = n
        operator = e
    else:
        print("Please specify the operation as x = e / d")
        return
    #print("\n N Bits", mp.bit_length(N))    
    blockLen = calcBlockSize(N)
    #print("Block Length", blockLen, type(blockLen))
    msgBlocks = getMsgBlocks(text, blockLen)
    if(len(msgBlocks[-1]) != blockLen):
        for i in range(blockLen - len(msgBlocks[-1])):
            msgBlocks[-1] += numToChars[np.random.randint(0, 2)]
            
    encryptedMsg = []
    for block in msgBlocks:
        msg = 0
        for i in range(blockLen):
            msg += mp.mul(charsToNum[block[i]], mp.mpz(charSpace)**(blockLen -1 -i))   
        #print("\n Orig Msg Bits", mp.bit_length(msg))
        #print("\n Orig Msg\n", msg)
        #print("\nOperator e\n", operator)
        eMsg = mp.powmod(mp.t_mod(msg, N), operator, N)
        encryptedMsg.append(eMsg)
    
    #print("\nencryptedMsgBlock", encryptedMsg)
    eMsg_Eng = ""
      
    enMsg = ""
    for msg in encryptedMsg:
        #qdash, remainder = mp.t_divmod(msg, mp.mpz(charSpace)**(blockLen))
        #print("\nQDash", qdash)
        remainder = msg
        m = []
        for i in range(blockLen):
            quot, remainder = mp.t_divmod(remainder, mp.mpz(charSpace)**(blockLen -1 -i))
            m.append(numToChars[quot])
        enMsg += ''.join(m)
        #print("EMsg\n", ''.join(m))
    
    return enMsg
        
def deRSA(text, sender, receiver, operation):
    '''RSA Decryption'''    
    
    operator = 0
    
    if(operation == "d"):
        sk = readFile("secret_" + receiver + ".txt").split()
        if validateCASignature(sk):
            print("\nCA SIGNATURE Verified. Hence Key is Valid")
        else:
            print("\nCA SIGNATURE Verification Failed. Hence Key is invalid")
            return
        print("\nDecrypting by Receiver's[{}] Secret Key.".format(receiver))    
        d = unsignDigital_CA(mp.mpz(sk[0].strip()))
        n = unsignDigital_CA(mp.mpz(sk[1].strip()))
        operator = d
        N = n
    elif(operation == "e"):
        pk = readFile("public_directory/public_" + sender + ".txt").split()
        if validateCASignature(pk):
            print("\nCA SIGNATURE Verified. Hence Key is Valid")
        else:
            print("\nCA SIGNATURE Verification Failed. Hence Key is invalid")
            return
        print("\nDecrypting by Sender's[{}] Public Key.".format(sender))
        e = unsignDigital_CA(mp.mpz(pk[0].strip()))
        n = unsignDigital_CA(mp.mpz(pk[1].strip()))
        operator = e
        N = n
    else:
        print("Please specify the operation as x = e / d")
        return
    blockLen = calcBlockSize(N)
    msgBlocks = getMsgBlocks(text, blockLen)
    #print("\nMSG BLOCKS DE RECEIVED\n", msgBlocks[0])
    
    decryptedMsg = []
    for block in msgBlocks:
        msg = 0
        if len(block) != 0:
            for i in range(blockLen):
                msg += mp.mul(charsToNum[block[i]], mp.mpz(charSpace)**(blockLen -1 -i))
            
            #msg += mp.mpz(charSpace)**(blockLen-1)
        
            decryptedMsg.append(msg)
    deMsgBlocks = []  
    tempDeMsg = ""
    
    
    for block in decryptedMsg:
        #print("\nBroken Blocks:\n" ,block)
        #print("\n nBroken Msg Bits", mp.bit_length(block))
        #print("\nOperator d\n", operator)
        #print("\n N: \n", N )
        remain = mp.powmod(mp.t_mod(block, N), operator, N)
        deMsgBlocks.append(remain)
        #print("After Decryptoin\n", remain)
      
    decryptedMsg = ""
    for msg in deMsgBlocks:
        remainder = msg
        m = []
        for i in range(blockLen):
            quot, remainder = mp.t_divmod(remainder, mp.mpz(charSpace)**(blockLen -1 -i))
            m.append(numToChars[mp.t_mod(quot, charSpace)])
        decryptedMsg += ''.join(m)
        
    return decryptedMsg


def prepareMsgForRSA(msgEnVig):
    '''
    Prepare msg for RSA Encryption by appending 
    "Length of Vigenere Cipher" + "Vigenere Key" + "Vigenere Encrypted Message
    '''
    return numToChars[vigKeyLen] + vigKey + msgEnVig

def recoverMsgForVigenere(text):
    '''Recover Vigenere Key and Rest of the Encrypted Message'''
    viglen = charsToNum[text[0]]
    vigKey = ""
    for i in range(viglen):
        vigKey += text[i+1]
    print("\nVigKey Recovered from Encrypted Message:", vigKey)
    restMsg = text[viglen+1:]
    #print(restMsg)
    return vigKey, restMsg

def sendSecureMessage(msg, receiver, sender):
    '''Encrpyt message by vigenere cipher'''
    MsgEn_Vig = enVigenere(msg, vigKey)
    
    '''Prepare Message for RSA = "Length of Vigenere Cipher" + "Vigenere Key" + "Vigenere Encrypted Message"'''
    MsgForRSA = prepareMsgForRSA(MsgEn_Vig)
    
    '''Encrpyt vigenere encrypted message from SECRET KEY of SENDER'''
    MsgEn_SenderPvtKey = enRSA(MsgForRSA, sender, receiver, "d") #d stands for Decrypt Operation with secret key
    
    '''Encrpyt the whole message with PUBLIC KEY of RECEIVER'''
    MsgEn_ReceiverPubKey = enRSA(MsgEn_SenderPvtKey, sender, receiver, "e") #e stands for Encrypt Operation with public key
    
    writeFile("sent_message.txt", MsgEn_ReceiverPubKey)
    return

def receiveAndDecryptMessage(msg, receiver, sender):
    '''Read Secure Message'''
    msgReceived = readFile("sent_message.txt")
    
    '''Decrpyt the message with SECRET KEY of RECEIVER'''
    MsgDe_ReceiverPvtKey = deRSA(msgReceived, sender, receiver, "d")
    
    '''Decrpyt the message with PUBLIC KEY of SENDER'''
    MsgDe_SenderPubKey = deRSA(MsgDe_ReceiverPvtKey, sender, receiver, "e")
    
    '''Recover Vigenere Key and Rest of the Encrypted Message'''
    vigkey, MsgEn_Vig = recoverMsgForVigenere(MsgDe_SenderPubKey)
    
    '''Decrypt with Vigenere Key'''
    MsgDe_Vig = deVigenere(MsgEn_Vig, vigkey)
    
    '''Write Message to the received message file'''
    writeFile("received_message.txt", MsgDe_Vig)
    
    print("\nRecovered Message:\n", MsgDe_Vig)
    
    return


if __name__ == "__main__":
    '''Initialize Variables'''
    charsToNum = charsDictGen() #Generate Characters to Number Dictionary
    numToChars = numbersDictGen() #Generate Numbers to Characters Dictionary
    vigKey = readFile("vigenere_key.txt") #read vigenere cipher key 
    vigKeyLen = len(vigKey)

    '''Publish CA Signed Public Keys in Public Directory and Send Secret Keys to respective users'''
    publishCaSignedKeys() #Publish Private and Public Keys signed by CA

    '''Take input from User'''
    Message = readFile("message.txt") #Read Msg from message text file
    SendTo = str(input("SEND MESSAGE TO: (a or b)")) #Input Receiver's Name
    From = str(input("FROM: (a or b)")) #Input Sender's Name

    '''Send Secure Message'''
    sendSecureMessage(Message, SendTo, From) #Send Message With High Security

    '''Receive and Decrypt Secure Message'''
    receiveAndDecryptMessage(Message, SendTo, From)

SEND MESSAGE TO: (a or b)a
FROM: (a or b)b

CA SIGNATURE Verified. Hence Key is Valid

Encrypting by Sender's[b] Secret Key.

CA SIGNATURE Verified. Hence Key is Valid

Encrypting by Receiver's[a] Public Key.

CA SIGNATURE Verified. Hence Key is Valid

Decrypting by Receiver's[a] Secret Key.

CA SIGNATURE Verified. Hence Key is Valid

Decrypting by Sender's[b] Public Key.

VigKey Recovered from Encrypted Message: thisisthevigenerecipherkey

Recovered Message:
 in another flashback, two fighter jets fly overhead and bomb the familys town. they rush to a nearby abandoned shelter as the rest of the towns population is decimated by the military. claire accidentally cuts herself on a rusty piece of iron as she descends into the shelter and notices the flesh around the cut is discolored and her blood black. they deduce that the virus is airborne and they have already been infected. harry potter is the kind of timeless literary achievement that comes around once in a lifetime. since the books